In [1]:
import sys
sys.path.append('../')

import json
# from datetime import datetime, timedelta
from instruments.swap import *
from datetime import datetime, timedelta
import pandas as pd

## Market Curves

In [2]:
class Market:
    def __init__(self) -> None:
        self.market_dfs = pd.read_csv('../data/markets/Market_2023_09_29_DFs.csv', 
                                        skiprows= 2,
                                        on_bad_lines='skip')
        self.evaluation_date = date(2023,9,29)
    @property
    def estr_curve(self):
        df_eur = self.market_dfs[self.market_dfs['Property'].str.contains("IR.EUR-ESTR-ON.DF")][['Property', 'Label']]
        dates = []
        for i in range(len(df_eur['Property'].values)):
            str_temp = df_eur['Property'].values[i][18:29]
            dates.append(datetime.strptime(str_temp, '%d-%b-%Y') )
        dfs_pillars = [(dates[t]-dates[0]).days/365 for t in range(len(dates))]
        dfs_market = df_eur['Label'].values
        return DiscountCurveSimple(dfs_pillars, dfs_market)
    
    @property
    def eur_6m_index(self):
        index_eur_6m = CurveRateIndex("EUR6M", "calendar", self.estr_curve, "6M")
        index_eur_6m.add_fixing(date(2023,6,30), 0.03)
        return index_eur_6m 

## Trades

In [3]:
md = Market()
with open('../data/trades/TradesAggregated_2.json', 'r') as file:
    trade_data = json.load(file)

In [4]:
for trade_json in trade_data:
    if trade_json['ref'] == '0-17092454':
        trade = trade_json
trade['data']['cpty']

'ATLANDES_SA'

In [5]:
def make_swap(trade):
    leg1_schedule_len = len(trade['data']['Leg1.schedule']['rows'])
    leg2_schedule_len = len(trade['data']['Leg2.schedule']['rows'])

    leg1_is_fixed = trade['data']['Leg1.isFixedLeg']
    leg2_is_fixed = trade['data']['Leg2.isFixedLeg']

    if not leg1_is_fixed:
        leg1_index = trade['data']['Leg1.index']
    elif not leg2_is_fixed:
        leg2_index = trade['data']['Leg2.index']

    date_format = '%Y-%m-%d'
    leg1_payment_dates = [trade['data']['Leg1.schedule']['rows'][i]['paymentDate'] for i in range(leg1_schedule_len)]
    leg1_payment_dates = [datetime.strptime(dt, date_format).date() for dt in leg1_payment_dates]
    leg2_payment_dates = [trade['data']['Leg2.schedule']['rows'][i]['paymentDate'] for i in range(leg2_schedule_len)]
    leg2_payment_dates = [datetime.strptime(dt, date_format).date() for dt in leg2_payment_dates]

    leg1_notionals = [trade['data']['Leg1.notionalTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_notionals = [trade['data']['Leg2.notionalTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_margins = [trade['data']['Leg1.marginTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_margins = [trade['data']['Leg2.marginTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_rate_factor = [trade['data']['Leg1.rateFactorTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_rate_factor = [trade['data']['Leg2.rateFactorTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_fixed_payments = [trade['data']['Leg1.fixedPaymentAmountTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_fixed_payments = [trade['data']['Leg2.fixedPaymentAmountTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]


    if leg1_is_fixed and not leg2_is_fixed:

        if leg2_index == 'EUR-EURIBOR-6M':
            index = md.eur_6m_index

        swap_obj = SwapFixedFloating(leg1_payment_dates,
                                    leg2_payment_dates,
                                    leg1_notionals,
                                    leg2_notionals ,
                                    leg2_rate_factor, # moltiplicatore variabile
                                    leg2_margins, # spread variabile
                                    index,
                                    leg1_margins) # tasso fisso
        
    elif leg2_is_fixed and not leg1_is_fixed:
        if leg1_index == 'EUR-EURIBOR-6M':
            index = md.eur_6m_index

        swap_obj = SwapFixedFloating(leg1_payment_dates,
                                    leg2_payment_dates,
                                    leg1_notionals,
                                    leg2_notionals ,
                                    leg1_rate_factor, # moltiplicatore variabile
                                    leg1_margins, # spread variabile
                                    index,
                                    leg2_margins) # tasso fisso
    return swap_obj

In [21]:
portfolio = {}
errors = []
for trade_json in trade_data:
    try:
        if trade_json['data']['productType'] == 'InterestRateSwapScript':
            portfolio['swap'] = {
                                f'ref{i+1}': trade_json['ref'],
                                'trade_object': make_swap(trade_json)}
            i = i+1
    except:
        portfolio['errors'] = {
                                f'ref': errors.extend(trade_json['ref'])}
        pass

In [24]:
dict_test = {}


In [27]:
dict_test['swap'] = [
    {
        ['ref': '123',
        'trade': 'oggetto']

    }
    
    ]


In [28]:
dict_test

{'swap': 2}

In [18]:
for trade_json in trade_data:
    print(trade_json['ref'])
    print(trade_json['data']['productType'] )

0-1401282
FXForward
0-1404296
FXForward
0-1381305
FXForward
0-1401287
FXForward
0-7563295
FXSwap
0-1379353
FXForward
0-1404303
FXForward
0-1404297
FXForward
0-1379359
FXForward
0-7563300
FXSwap
0-1401281
FXForward
0-1399744
FXForward
0-7575331
FXSwap
0-1381296
FXForward
0-1401288
FXForward
0-7596659
FXSwap
0-94515773
InterestRateSwapScript
0-1377957
FXForward
0-1379365
FXForward
0-1377960
FXForward
0-1404286
FXForward
21644718-35795858_phase1
CapFloor
21644718-35795858_phase2
CapFloor
21644718-35795858_phase3
CapFloor
21644718-35795858_phase4
CapFloor
21644718-35795858_phase5
CapFloor
21644718-35795858_phase6
CapFloor
21644718-35795858_phase7
CapFloor
21644718-35795858_phase8
CapFloor
21644718-35795858_phase9
CapFloor
21644718-35795858_phase10
CapFloor
21644718-35795858_phase11
CapFloor
15453171-28319781
InterestRateSwapScript
15453225-28319834
InterestRateSwapScript
15525550-28390732
InterestRateSwapScript
15525556-28390737
InterestRateSwapScript
0-1399499
FXForward
0-1393897
FXForwar

KeyError: 'productType'

In [8]:
# funzione per trade
trade_ref = trade['ref']
if trade['data']['productType'] == 'InterestRateSwapScript':
    # chiama funzione per swap
    swap_obj = make_swap(trade)


In [12]:
npv = swap_obj.price(md.estr_curve, md.evaluation_date) 
npv_float = swap_obj.floating_leg.npv(md.estr_curve, md.evaluation_date) 
npv_fixed = swap_obj.fixed_leg.npv(md.estr_curve, md.evaluation_date) 

print(f"NPV TQ: {npv :,.0f}")
print(f"NPV FixedLeg TQ: {npv_fixed :,.0f}")
print(f"NPV FloatingLeg TQ: {npv_float :,.0f}")

NPV TQ: -10,478,948
NPV FixedLeg TQ: -46,972,945
NPV FloatingLeg TQ: 36,493,997
